In [1]:
print("Importing libraries")

# import packages 
import os
import tempfile

#import anndata
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scanpy as sc
import celltypist
import time

print("Reading in datasets")




Importing libraries
Reading in datasets


In [2]:
adata_all=sc.read_h5ad('/nfs/team298/sm54/BoneAtlasProject/data/cross_organ_haem/myeloid_lineage/latent20_layers2_hidden128_hvg2000/FULL_scvi.h5ad')

In [3]:
adata_all.obs['published_propagated_anno_harmonised_LEVEL1_V8'].value_counts()

published_propagated_anno_harmonised_LEVEL1_V8
MACROPHAGE       207796
MONOCYTE          70027
HSC/MPP & PRO     29357
DC                27141
EO/BASO/MAST      13014
NEUTROPHIL         7702
MYELOCYTE          7334
PROMYELOCYTE       3873
OSTEOCLAST         1400
Name: count, dtype: int64

In [4]:
adata_all.obs['Age_numeric_str'].value_counts()

Age_numeric_str
14      64195
12      41765
6       41189
17      36039
7       30444
15      30336
9       28263
5       19819
16      19128
11      15668
10      13187
2       10909
13       6062
8        5571
19       4812
<NA>      257
Name: count, dtype: int64

In [5]:
adata_all=adata_all[adata_all.obs['Age_numeric_str']!='<NA>']

In [8]:
adata_all.obs['Age_numeric_str'].value_counts()

Age_numeric_str
14    64195
12    41765
6     41189
17    36039
7     30444
15    30336
9     28263
5     19819
16    19128
11    15668
10    13187
2     10909
13     6062
8      5571
19     4812
Name: count, dtype: int64

In [9]:
adata_all.obs['Anatomical_Site_V2'].value_counts()

Anatomical_Site_V2
Liver        97888
Long Bone    72427
Skin         61433
Yolk Sac     48020
Spleen       26729
Spine        21189
Brain        11173
Skull         7299
Kidney        6997
Gut           5595
Meninges      5193
Thymus        3444
Name: count, dtype: int64

In [10]:
adata_all.obs['Organ_Celltype_Age']= adata_all.obs['Anatomical_Site_V2'].astype(str).copy() + '_' + adata_all.obs['published_propagated_anno_harmonised_LEVEL1_V8'].astype(str).copy() + '_' + adata_all.obs['Age_numeric_str'].astype(str).copy()

In [ ]:
adata_all.obs['Organ_Celltype_Age'].value_counts()

In [12]:
adata_all.obs['Organ_Celltype_Age'] = adata_all.obs['Organ_Celltype_Age'].astype('category')


In [13]:
vc = adata_all.obs['Organ_Celltype_Age'].value_counts()


In [ ]:
adata_all.obs['Anatomical_Site_V2'].value_counts()

In [22]:
keep_groups = vc[vc >= 200].index


In [23]:
adata_all2=adata_all[adata_all.obs['Organ_Celltype_Age'].isin(keep_groups)]

In [24]:
adata_all2.obs['Organ_Celltype_Age'].value_counts()

Organ_Celltype_Age
Yolk Sac_MACROPHAGE_6    21758
Liver_MACROPHAGE_9       13684
Skin_MACROPHAGE_14       12585
Yolk Sac_MACROPHAGE_7    11800
Spleen_MACROPHAGE_14     11401
                         ...  
Skin_DC_16                 205
Spine_MACROPHAGE_10        205
Spleen_MONOCYTE_16         204
Liver_EO/BASO/MAST_14      200
Liver_EO/BASO/MAST_6       200
Name: count, Length: 211, dtype: int64

In [26]:
adata_all2.write_h5ad('/nfs/team298/sm54/BoneAtlasProject/data/cross_organ_haem/myeloid_lineage/celltypist_training.h5ad')

In [ ]:
adata_all2

In [ ]:
adata_all2.X= adata_all2.layers["raw_counts"].copy()

# adata_ref.X.max()

print("normalising")

sc.pp.normalize_total(adata_all2, target_sum = 1e4)
sc.pp.log1p(adata_all2)



In [ ]:
np.expm1(adata_all2.X).sum(axis=1)


In [ ]:

np.expm1(adata_ref.X).sum(axis=1)

print("Training with SGD for feature selection...")




# Use `celltypist.train` to quickly train a rough CellTypist model.
# You can also set `mini_batch = True` to enable mini-batch training.
t_start = time.time()
model_fs = celltypist.train(adata_ref, 'cell.labels', n_jobs = 10, max_iter = 6, use_SGD = True)
t_end = time.time()
print(f"Time elapsed: {t_end - t_start} seconds")



#  Model training with downsampled genes 

gene_index = np.argpartition(np.abs(model_fs.classifier.coef_), -600, axis = 1)[:, -600:]
gene_index = np.unique(gene_index)


print(f"Number of genes selected: {len(gene_index)}")


print("Training with reduced features...")

# Add `check_expression = False` to bypass expression check with only a subset of genes.
t_start = time.time()
model = celltypist.train(adata_ref[:, gene_index], 'cell.labels', check_expression = False, n_jobs = 10, max_iter = 100)
t_end = time.time()
print(f"Time elapsed: {(t_end - t_start)/60} minutes")

model.write('/nfs/team298/sm54/BoneAtlasProject/metadata/celltypist_models/macs_ys.pkl')